<a href="https://colab.research.google.com/github/MaheshbabuAmuda/NNDL/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment-2**

-20K45A0229.

#Import required libraries

In [15]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

##Import Data

In [16]:
data = pd.read_excel('/content/Load Data in kW.xlsx')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


#Data normalization

In [17]:
x = []
y = []
for i in range(24 * 7, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i - 24 * 7])
    y.append(data['Load (kW)'].iloc[i])
data = pd.DataFrame({'Previous Week Load' : x, 'Present Week Load' : y})
data.rename(columns={'Previous Week Load': 'X',
                   'Present Week Load': 'Y'},
          inplace=True)
data.head()

,X,Y
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


In [19]:
data = pd.DataFrame({'Previous Day Load' : x, 'Present Day Load' : y})
data.head()

,Previous Day Load,Present Day Load
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


##Data split to training and testing

In [20]:
maxX = np.max(data['Previous Day Load'])
minX = np.min(data['Previous Day Load'])
maxY = np.max(data['Present Day Load'])
minY = np.min(data['Present Day Load'])

data['Previous Day Load'] = (data['Previous Day Load'] - minX) / (maxX - minX)
data['Present Day Load'] = (data['Present Day Load'] - minY) / (maxY - minY)

In [21]:
x = data['Previous Day Load']
y = data['Present Day Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

##Momentum + SGD Algorithm

In [22]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 200
vm = 0
vc = 0
v = 0.5

In [24]:
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
       
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
    
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        
        m += vm
        c += vc

print(f'm = {m}\nc = {c}')

m = 0.4690814435573055
c = 0.20813090203476736


##Model prediction on training & testing  data

In [25]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

In [27]:
#Training Accuracies
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'Mean absolute error = {training_MAE}')
print(f'Mean square error = {training_MSE}')
print(f'Root mean square error = {training_RMSE}')

Training Error :
Mean absolute error = 751.847651951687
Mean square error = 936265.0638764747
Root mean square error = 967.6079081303928


In [28]:
#Testing Accuracies
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'Mean absolute error = {testing_MAE}')
print(f'Mean square error = {testing_MSE}')
print(f'Root mean square error = {testing_RMSE}')

Testing Error :
Mean absolute error = 810.437330174427
Mean square error = 1039598.3374445771
Root mean square error = 1019.6069524304829


##Deployment

In [29]:
x = float(input('Enter the load at present hour on previous day : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load at present hour on previous day : 5000
Predicted load at present hour : 5274.902299073447
